In [121]:
'''
Install dependencies
'''
!pip3 install -U spacy contractions beautifulsoup4 spacy-cleaner pyspellchecker requests
!python3 -m spacy download en_core_web_lg

  Using cached spacy-3.5.3-cp310-cp310-macosx_11_0_arm64.whl (6.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 4.3 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [129]:
'''
Import statements
'''
import re
from bs4 import BeautifulSoup
import contractions
from spellchecker import SpellChecker
import en_core_web_lg
import spacy
import requests

In [212]:
'''
Text pre-processing
'''
def preprocess(review: str) -> list:
    # HTML decoding
    review = BeautifulSoup(review).get_text()
    # Convert words to lowercase
    review = review.lower()
    # Expand contractions
    review_list = []
    for word in review.split():
        review_list.append(contractions.fix(word))
    # Make entities a single token eg. New York -> New-York
    nlp = en_core_web_lg.load()
    entity_list = []
    for ele in nlp(review).ents:
        if len(str(ele.text).split()) > 1:
            entity_list.append((ele.text, str(ele.text).replace(' ','-')))
    for item in entity_list:
        review = review.replace(item[0], item[1])
    # Digits and punctuations removal
    review = ' '.join(re.sub('[^a-zA-Z-]', ' ', review).split())
    review_list = re.sub('-', '', review).split()
    # Correct slang words / phrases
    abbr_dict = {}
    def getAbbr(alpha, abbr_dict):
        res = requests.get('https://www.noslang.com/dictionary/'+alpha)
        soup = BeautifulSoup(res.text, 'html.parser')
        for abbr in soup.find_all('abbr'):
            full = abbr['title'].lower()
            abrv = abbr.find('dt').text[:-2]
            abbr_dict[abrv] = full
    abbr_list = []
    for char in range(97,123):
        abbr_list.append(chr(char))
    for i in abbr_list:
        getAbbr(i, abbr_dict)
    expanded = {}
    for word in review_list:
        if word in abbr_dict:
            expanded[word] = abbr_dict[word]
    review = ' '.join(review_list)
    for item in expanded:
        review = review.replace(item, expanded[item])
    review_list = review.split()
    # Correct spelling
    corrector = SpellChecker()
    misspelled = corrector.unknown(review_list)
    corrected = {}
    for word in misspelled:
        if corrector.correction(word):
            corrected[word] = corrector.correction(word)
    review = ' '.join(review_list)
    for item in corrected:
        review = review.replace(item, corrected[item])
    # Tokenization
    doc = nlp(review)
    # Word tokenization and stop word removal
    tokenized_word_list = [token.text for token in doc if not token.is_stop and len(token.text)>1]
    '''To consider: sentence tokenization'''
    return tokenized_word_list